In [7]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [8]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [9]:


# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001


In [10]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=False)

test_dataset = torchvision.datasets.MNIST(root='./data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [11]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            #nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding) 定义了一个二维卷积操作。
            #其中，in_channels 表示输入通道数，out_channels 表示输出通道数，kernel_size 表示卷积核的大小，stride 表示卷积步长，padding 表示填充大小。
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            #定义了一个批量归一化操作。其中，num_features 表示输入特征数
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        #全连接层，计算，L1:（28+2*2-（5-1））/2=14   L2:(14+2*2-(5-1))/2=7
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        #在神经网络中，不同类型的层接受不同形状的输入数据。
        #例如，卷积层通常接受一个四维张量作为输入，其中第一维表示批量大小，第二维表示通道数量，第三维和第四维表示高度和宽度。
        #而全连接层则接受一个二维张量作为输入，其中第一维表示批量大小，第二维表示特征数量。
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out


In [12]:


model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/600], Loss: 0.1108
Epoch [1/5], Step [200/600], Loss: 0.0912
Epoch [1/5], Step [300/600], Loss: 0.0290
Epoch [1/5], Step [400/600], Loss: 0.0283
Epoch [1/5], Step [500/600], Loss: 0.0793
Epoch [1/5], Step [600/600], Loss: 0.0869
Epoch [2/5], Step [100/600], Loss: 0.1001
Epoch [2/5], Step [200/600], Loss: 0.0544
Epoch [2/5], Step [300/600], Loss: 0.0799
Epoch [2/5], Step [400/600], Loss: 0.0367
Epoch [2/5], Step [500/600], Loss: 0.1118
Epoch [2/5], Step [600/600], Loss: 0.0870
Epoch [3/5], Step [100/600], Loss: 0.0152
Epoch [3/5], Step [200/600], Loss: 0.0359
Epoch [3/5], Step [300/600], Loss: 0.0827
Epoch [3/5], Step [400/600], Loss: 0.0620
Epoch [3/5], Step [500/600], Loss: 0.0341
Epoch [3/5], Step [600/600], Loss: 0.0242


In [ ]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')